# 运输问题 例2 p. 90

例 2 设有三个化肥厂( A, B, C)供应四个地区(I,II,III,IV)的农用化肥。假定等 量的化肥在这些地区使用效果相同。各化肥厂年 产量 , 各 地区 年需要 量及 从各化 肥厂 到 各地区运送单位化肥的运价如表 3 -25 所示。试求出总的运费最节省的化肥调拨方案。

参考：`facility.py` http://www.gurobi.com/documentation/7.0/examples/facility_py.html

### Step 1: Import functions from the gurobipy module

In [9]:
from gurobipy import *


### Step 2: Create empty model

In [10]:
m = Model("facility")

# Warehouse demand in thousands of units
# 销量
# 最大销量
demand_min = [30, 70, 0, 10]
# 最小销量
demand_max = [50, 70, 30, GRB.INFINITY]
demand = demand_min  # 便于 len(demand) 取数组大小

# Plant capacity in thousands of units
# 产量
capacity = [50, 60, 50]

# Fixed costs for each plant
# 无固定支出
fixedCosts = [0, 0, 0, 0]

# Transportation costs per thousand units
transCosts = [[16, 13, 22 , 17],
              [14, 13, 19 , 15],
              [19, 20, 23 , GRB.INFINITY],
             ]

transCostsDict = {}


### Step 3: Create activitiy variables

In [11]:
# Range of plants and warehouses
plants = range(len(capacity))
warehouses = range(len(demand))

# 将cost矩阵转为字典，预备计算代价
for i in range(len(capacity)):
    for j in range(len(demand)):
        transCostsDict[(i,j)] = transCosts[i][j]
print "cost矩阵", transCosts
print "cost字典()", transCostsDict
##
# facility.py 中 考虑工厂开放存在cost，增加关闭变量
# Plant open decision variables: open[p] == 1 if plant p is open.
# open = m.addVars(plants,
#                  vtype=GRB.BINARY,
#                  obj=fixedCosts,
#                  name="open")
# 
## 我们这里假设所有工厂都在开放状态
open = [1] * len(plants)

# Transportation decision variables: transport[w,p] captures the
# optimal quantity to transport to warehouse w from plant p
# transport = m.addVars(warehouses, plants, obj=transport name="trans")
# 将原有的 obj=transport 修改为 obj 函数； 
# 按原有的写法 obj 不起作用; 与答案第一行一致，但是二三行不一样。 参答案 p. 83 表 3-13
transport = m.addVars(plants, warehouses, name="trans")
transport

cost矩阵 [[16, 13, 22, 17], [14, 13, 19, 15], [19, 20, 23, 1e+100]]
cost字典() {(0, 1): 13, (1, 2): 19, (0, 0): 16, (2, 1): 20, (0, 2): 22, (2, 0): 19, (1, 3): 15, (2, 3): 1e+100, (2, 2): 23, (1, 0): 14, (0, 3): 17, (1, 1): 13}


{(0, 0): <gurobi.Var trans[0,0]>,
 (0, 1): <gurobi.Var trans[0,1]>,
 (0, 2): <gurobi.Var trans[0,2]>,
 (0, 3): <gurobi.Var trans[0,3]>,
 (1, 0): <gurobi.Var trans[1,0]>,
 (1, 1): <gurobi.Var trans[1,1]>,
 (1, 2): <gurobi.Var trans[1,2]>,
 (1, 3): <gurobi.Var trans[1,3]>,
 (2, 0): <gurobi.Var trans[2,0]>,
 (2, 1): <gurobi.Var trans[2,1]>,
 (2, 2): <gurobi.Var trans[2,2]>,
 (2, 3): <gurobi.Var trans[2,3]>}

### Step 4: Set objective function

In [12]:
# 为何这里的 obj=transCosts ?
# transport = m.addVars(warehouses, plants, name="trans")
obj = transport.prod(transCostsDict)
m.setObjective( obj , GRB.MINIMIZE )
# m.modelSense = GRB.MINIMIZE
# m.setObjective(quicksum(transport.prod(transCosts)), GRB.MINIMIZE)
print "目标函数 obj:", obj

目标函数 obj: <gurobi.LinExpr: 16.0 trans[0,0] + 13.0 trans[0,1] + 22.0 trans[0,2] + 17.0 trans[0,3] + 14.0 trans[1,0] + 13.0 trans[1,1] + 19.0 trans[1,2] + 15.0 trans[1,3] + 19.0 trans[2,0] + 20.0 trans[2,1] + 23.0 trans[2,2] + 1e+100 trans[2,3]>


### Step 5: Add constraints

In [13]:
# Production constraints
# Note that the right-hand limit sets the production to zero if the plant
# is closed
# <= 修改为 ==
m.addConstrs(
    (transport.sum(p, '*') == capacity[p] for p in plants),
    "Capacity")

# Demand constraints
m.addConstrs(
    (transport.sum('*', w) <= demand_max[w] for w in warehouses),
    "Demand")

m.addConstrs(
    (transport.sum('*', w) >= demand_min[w] for w in warehouses),
    "Demand")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>}

### Step 6: Solve model

In [14]:
m.optimize()

Optimize a model with 11 rows, 12 columns and 36 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 1e+100]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 7e+01]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 5 rows and 2 columns
Presolve time: 0.04s
Presolved: 6 rows, 11 columns, 20 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.3800000e+03   5.000000e+01   0.000000e+00      0s
       3    2.4600000e+03   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.08 seconds
Optimal objective  2.460000000e+03


### Step 8: Print variable values for optimal solution

In [18]:
# Print solution
print('\nTOTAL COSTS: %g' % m.objVal)
print('SOLUTION: 运输矩阵解为')
print "   ",
for w in warehouses:
    print "[B{0}]".format(w),
print 
for p in plants:
    print "[A{0}]".format(p), 
    if open[p] > 0.99:
        # print('Plant %s open' % p)
        for w in warehouses:
            #if transport[w,p].x > 0:
            #    print('  Transport %g units to warehouse %s' % \
            #          (transport[w,p].x, w))
            print transport[p,w].x, 
        print
    else:
        print('Plant %s closed!' % p)
        
# print transport[0,1].x
# print transport[1,0].x
# print transport[3,0].x
print "\n最优值为：", m.ObjVal


TOTAL COSTS: 2460
SOLUTION: 运输矩阵解为
    [B0] [B1] [B2] [B3]
[A0] 0.0 50.0 0.0 0.0
[A1] 0.0 20.0 0.0 40.0
[A2] 50.0 0.0 0.0 0.0

最优值为： 2460.0


```
# 解出的最优解满足生产运输约束, 与 p.91 表 3-28 答案一致
```